# NAS_CNN
## Pipelines

In [1]:
#Imports
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from evolutionary_search import EvolutionaryAlgorithmSearchCV
from sklearn.datasets import fetch_openml
from sklearn.impute import SimpleImputer
from sklearn.linear_model import SGDClassifier, LogisticRegression, Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV #, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelBinarizer, LabelEncoder, MaxAbsScaler, MinMaxScaler
from sklearn.svm import SVR, SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

from tensorflow.python.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.losses import MSE, binary_crossentropy, categorical_crossentropy, sparse_categorical_crossentropy, kl_divergence
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.python.keras import Input
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.wrappers.scikit_learn import KerasClassifier

In [2]:
#MNIST Data_id = 554
#Fashion-MNIST Data_id = 40996
#CIFAR-10 Data_id = 40927
X,y = fetch_openml(data_id=554,return_X_y=True)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=.8)

c:\users\caleb\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2178: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [ ]:
#Wenn es sich um ein Bild mit drei Kanäle (Farbbild) handelt, dann Farbe=True
farbe = True


#Preprocessing
from sklearn.preprocessing import MinMaxScaler, Binarizer

if farbe:
    scaler = StandardScaler()
    P = scaler.fit_transform(X)
else:
    scaler = StandardScaler()
    P = scaler.fit_transform(X)
    
size = list(X.shape)
def Farbe(XX, farbe, Bild_size):
    if farbe:
        X = XX.reshape(len(y), 3, np.sqrt(Bild_size/3).astype(int), np.sqrt(Bild_size/3).astype(int)).transpose(0,2,3,1).astype('int32')
    else:
        X = XX.reshape((len(y), 1, np.sqrt(Bild_size).astype(int), np.sqrt(Bild_size).astype(int))).transpose(0,2,3,1).astype('int32')
    return X



P = Farbe(XX = P, farbe = farbe, Bild_size = size[1])
y = y.astype(int)
X_train, X_test, y_train, y_test = train_test_split(P,y, train_size=.8)


### Plot einige Daten

In [ ]:
for i in range(10, 15):
    some_digit_train = X_train[i]
    plt.imshow(some_digit_train, cmap="binary")
    plt.title(y_train[i])
    plt.show()    

### Model

In [ ]:
if farbe:
    input_shape = list(P.shape)
    input_shape = (input_shape[1],input_shape[2],input_shape[3])
else:
    input_shape = list(P.shape)
    input_shape = (input_shape[1],input_shape[2],1)

In [4]:
def build_nn(activation = 'relu', 
             activation_end='softmax', 
             n_neurons_1=200, 
             n_neurons_2=100,
             n_conv=1,
             kernel_size=3,
             input_shape = input_shape,
             n_layers=1):
        # detect and init the TPU
#    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

    # instantiate a distribution strategy
 #   tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

    # instantiating the model in the strategy scope creates the model on the TPU
  #  with tpu_strategy.scope():
        model = Sequential()
        model.add(Conv2D(filters = 32, kernel_size=(kernel_size, kernel_size), padding="same", activation=activation, input_shape=input_shape, name="conv_1"))
        model.add(MaxPooling2D(pool_size=(2, 2), name="Max_Pooling_1"))
        for j in range(n_conv):
            model.add(Conv2D(filters = 32*2**(j), kernel_size=(kernel_size, kernel_size), padding="same", activation=activation))        
            model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Flatten(name="in_1_dim_vektor"))
        
        for i in range(n_layers):
            model.add(Dense(n_neurons_1,activation=activation))
        model.add(Dense(n_neurons_2,activation=activation, name="versteckte_Schicht"))
        model.add(Dense(10,activation=activation_end, name="Ausgang"))

        lr_schedule = ExponentialDecay(
        initial_learning_rate = 0.001,
        decay_steps=100000,
        decay_rate=0.96,
        staircase=True)

        model.compile(loss=sparse_categorical_crossentropy, 
                      optimizer= Adam(learning_rate = lr_schedule), 
                      metrics=["accuracy"])
        return model

In [ ]:
model = build_nn()
model.summary()

### Simple Pipeline

In [5]:
steps = [
    ('clf',  KerasClassifier(build_nn) )
]
pipe = Pipeline(steps=steps)

In [6]:
pipe.fit(X_train, y_train)

1750/1750 [==============================] - 15s 3ms/step - loss: 0.4831 - accuracy: 0.8669


Pipeline(memory=None,
     steps=[('preprocessing', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x000001C949207C48>)])

#### Simple Pipeline Score:

In [7]:
y_pred = pipe.predict(X_test)
accuracy_score(y_true=y_test,y_pred=y_pred)

c:\users\caleb\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.9494285714285714

### Pipeline Optimization

In [1]:
params = [
            {
            'clf__n_conv': [1, 2, 3, 4],
            'clf__activation': ['linear','tanh', 'relu', 'sigmoid'],
            'clf__n_layers': [0, 1, 2, 3],
            'clf__n_neurons_1': [200, 300, 400, 500],
            'clf__n_neurons_2': [50, 100, 150]
            }    
    ]

In [9]:
clf2 = RandomizedSearchCV(estimator=pipe, param_distributions=params, 
                          n_iter=15, cv=4, verbose =2)
%time clf2.fit(X=X_train,y=y_train)

Fitting 4 folds for each of 15 candidates, totalling 60 fits
[CV] clf__optimizer=<class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, clf__learning_rate=0.1, clf__activation=sigmoid 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


1313/1313 [==============================] - 2s 2ms/step - loss: 1.6789 - accuracy: 0.4787
[CV]  clf__optimizer=<class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, clf__learning_rate=0.1, clf__activation=sigmoid, total=  29.5s
[CV] clf__optimizer=<class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, clf__learning_rate=0.1, clf__activation=sigmoid 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   32.4s remaining:    0.0s


1313/1313 [==============================] - 4s 3ms/step - loss: 2.0638 - accuracy: 0.1878
[CV]  clf__optimizer=<class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, clf__learning_rate=0.1, clf__activation=sigmoid, total=  28.7s
[CV] clf__optimizer=<class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, clf__learning_rate=0.1, clf__activation=sigmoid 
1313/1313 [==============================] - 3s 2ms/step - loss: 1.4246 - accuracy: 0.4857
[CV]  clf__optimizer=<class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, clf__learning_rate=0.1, clf__activation=sigmoid, total=   8.7s
[CV] clf__optimizer=<class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, clf__learning_rate=0.1, clf__activation=sigmoid 
1313/1313 [==============================] - 3s 3ms/step - loss: 1.7497 - accuracy: 0.2702
[CV]  clf__optimizer=<class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, clf__learning_rate=0.1, clf__activation=sigmoid, total=   9.4s
[CV] clf__optimizer=<class 'tensorflow.python.k

1313/1313 [==============================] - 3s 2ms/step - loss: 0.3998 - accuracy: 0.8876
[CV]  clf__optimizer=<class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, clf__learning_rate=0.01, clf__activation=relu, total=   8.8s
[CV] clf__optimizer=<class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, clf__learning_rate=0.0001, clf__activation=sigmoid 
1313/1313 [==============================] - 3s 2ms/step - loss: 0.4930 - accuracy: 0.8966
[CV]  clf__optimizer=<class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, clf__learning_rate=0.0001, clf__activation=sigmoid, total=   8.6s
[CV] clf__optimizer=<class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, clf__learning_rate=0.0001, clf__activation=sigmoid 
1313/1313 [==============================] - 3s 2ms/step - loss: 0.5215 - accuracy: 0.8911
[CV]  clf__optimizer=<class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, clf__learning_rate=0.0001, clf__activation=sigmoid, total=  18.2s
[CV] clf__optimizer=<class 'tensorflo

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed: 14.9min finished


1750/1750 [==============================] - 9s 4ms/step - loss: 0.3415 - accuracy: 0.8983
Wall time: 15min 8s


RandomizedSearchCV(cv=4, error_score='raise-deprecating',
          estimator=Pipeline(memory=None,
     steps=[('preprocessing', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x000001F4A05DA3C8>)]),
          fit_params=None, iid='warn', n_iter=15, n_jobs=None,
          param_distributions={'clf__activation': ['linear', 'sigmoid', 'tanh', 'relu'], 'clf__learning_rate': [0.0001, 0.001, 0.01, 0.1], 'clf__optimizer': [<class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [ ]:
y_pred = clf2.predict(X_test)
accuracy_score(y_true=y_test,y_pred=y_pred)

### Auswertung

In [ ]:
clf2.best_params_, clf2.best_score_

In [ ]:
clf2.cv_results_